# Extracting Shirer components from fMRI data

We'll be working with the afni-smooth derivatives from the Nastase et. al. dataset. These niftis have been through the following preprocessing steps: 

#### fMRIPrep 20.0.5
* Motion correction: FSL MCFLIRT
* Slice timing correction: AFNI 3dTshift
* Susceptibility distortion correction: fieldmap-less method using BOLD-T1w registration with intensity inversion; ANTs antsRegistration
* Co-registration to T1w: boundary-based registration (bbregister, FreeSurfer)
* Spatial normalization: to MNI152NLin2009cAsym using the transforms from T1w → MNI
* Resampling: BOLD resampled to MNI space (volumetric) with 2 mm isotropic resolution (and fsaverage for surface)
* Confounds extraction: motion parameters (6 DOF), FD, DVARS, aCompCor (from WM and CSF masks), global signal, cosine drift terms
##### Note: No spatial smoothing, no global signal regression, no temporal filtering

#### Confound regression in AFNI
* Nuisance regression of the confounds listed above
* No smoothing

#### Smoothing in AFNI
* 6 mm FWHM

In [5]:
import os
import re
import pandas as pd
from glob import glob


##### Now, we will have to z-score voxelwise across time...

In [ ]:
# collect the paths of all the preprocessed, smoothed nifti images that we'll be working with

# z score voxelwise data across time 

##### Inspecting the Shirer masks...

In [6]:
# Set path to directory with Shirer masks
mask_dir = "/home/jovyan/shirer2012"

# List all mask files
mask_files = [f for f in os.listdir(mask_dir) if f.endswith("_mask.nii")]

# Extract metadata from filenames
records = []
for fname in mask_files:
    match = re.match(
        r"network-(?P<network>.+?)_space-MNI152NLin2009cAsym_res-(?P<res>\d{2})(?:_thresh-(?P<thresh>\d{2}))?_mask\.nii",
        fname
    )
    if match:
        records.append({
            "filename": fname,
            "network": match.group("network"),
            "resolution": f"res-{match.group('res')}",
            "thresholded": "thresh-" + match.group("thresh") if match.group("thresh") else "none"
        })
    else:
        print(f"[WARN] Filename did not match pattern: {fname}")

# Convert to DataFrame
df = pd.DataFrame(records)

# Display summary
summary = df.groupby("network")[["resolution", "thresholded"]].agg(lambda x: sorted(set(x)))
print("=== Shirer Mask Summary ===")
print(summary)

pattern = os.path.join(mask_dir, "*_res-02_thresh-01_mask.nii")

mask_paths = sorted(glob(pattern))

print("=== res-02_thresh-01 Shirer Masks ===")
for path in mask_paths:
    print(path)

=== Shirer Mask Summary ===
                         resolution        thresholded
network                                               
anteriorsalience   [res-01, res-02]  [none, thresh-01]
auditory           [res-01, res-02]  [none, thresh-01]
basalganglia       [res-01, res-02]  [none, thresh-01]
dorsaldmn          [res-01, res-02]  [none, thresh-01]
highervisual       [res-01, res-02]  [none, thresh-01]
language           [res-01, res-02]  [none, thresh-01]
leftcontrol        [res-01, res-02]  [none, thresh-01]
posteriorsalience  [res-01, res-02]  [none, thresh-01]
precuneus          [res-01, res-02]  [none, thresh-01]
primaryvisual      [res-01, res-02]  [none, thresh-01]
rightcontrol       [res-01, res-02]  [none, thresh-01]
sensorimotor       [res-01, res-02]  [none, thresh-01]
ventraldmn         [res-01, res-02]  [none, thresh-01]
visuospatial       [res-01, res-02]  [none, thresh-01]
=== res-02_thresh-01 Shirer Masks ===
/home/jovyan/shirer2012/network-anteriorsalience_space